In [1]:
import requests
import csv
import time
import random
from tqdm import tqdm
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.exceptions import RequestException
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import json
import openai

In [2]:
# Kindroid API setting
KINDROID_API_KEY = "kn_b238e912-d38b-40d2-b57f-1414fd7eb41e"
KINDROID_AI_ID = "mhfcLWFQuXSlxDWlVLAs"
KINDROID_BASE_URL = "https://api.kindroid.ai/v1"

# ChatGPT API setting
CHATGPT_API_KEY = "sk-oNNG4fOvuWOD3GgjqWlS6gBQtLh5nqCz38pjnR6kZwQgUgOn"
CHATGPT_BASE_URL = "https://yunwu.ai/v1"

In [3]:
# A list of sentences to end a conversation
farewell_messages = [
    "I think it's time for me to head out now.",
    "It seems like a good time to wrap things up.",
    "I probably should get going now.",
    "Let's go ahead and call it a day for now.",
    "I think I'm going to log off now.",
    "I'm gonna bounce out of this convo.",
    "I think I'm done chatting for now.",
    "I'll just leave things here for now.",
    "I'm going to go ahead and sign off.",
    "I'll catch you later; thanks for the chat!",
    "I'm stepping out for now.",
    "I think I'm going to tap out here.",
    "I'm just gonna close this chat now.",
    "That's enough chatting for me today.",
    "I'll stop here, thanks for everything!",
    "I'm going to head off now.",
    "I'll catch you next time we chat.",
    "That'll be it for me today!",
    "I'm good for now; thanks a lot!",
    "Wrapping things up here; talk soon!",
    "That's all from my end for now.",
    "I think I'll call it quits here.",
    "I'll go ahead and check out of this chat.",
    "I'm about to take off now.",
    "Guess I'll go ahead and head out.",
    "I think I'll dip out of the convo now.",
    "I'm going to go ahead and sign out.",
    "All set on my end, thanks a lot!",
    "I'll take a break from this chat.",
    "I think I'll log off now.",
    "I'm just going to step away for now.",
    "I'll catch you on the flip side!",
    "I think I'll go ahead and move on.",
    "Alright, it's time for me to peace out.",
    "This is where I'll wrap things up.",
    "I'll just leave it here for now.",
    "I'll go ahead and close out this chat.",
    "Heading out of the convo; thanks!",
    "I'm checking out of this conversation.",
    "I'll go ahead and bounce for now.",
    "I'm going offline now.",
    "Ready to wrap things up here.",
    "I'm going offline at this point.",
    "Thanks a lot; I think I'm all set now.",
    "I'm going to leave the chat for now.",
    "I'm good to wrap things up here.",
    "I'll go ahead and close out now.",
    "I'll go ahead and sign off here.",
    "I'm going to wrap up on my end.",
    "I'll be taking a break here. Thanks!"
]


In [4]:
class KindroidStillRespondingError(Exception):
    pass

@retry(stop=stop_after_attempt(10), 
       wait=wait_exponential(multiplier=1, min=2, max=60),
       retry=retry_if_exception_type(KindroidStillRespondingError))
def send_message_to_kindroid(ai_id, message):
    headers = {
        "Authorization": f"Bearer {KINDROID_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "ai_id": ai_id,
        "message": message
    }
    
    try:
        response = requests.post(f"{KINDROID_BASE_URL}/send-message", json=payload, headers=headers)
        response.raise_for_status()
        return response.text
    except RequestException as e:
        print(f"Error from Kindroid: {str(e)}")
        raise

@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=4, max=10))
def send_message_to_chatgpt(messages):
    client = openai.OpenAI(
        base_url=CHATGPT_BASE_URL,
        api_key=CHATGPT_API_KEY
    )
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=50,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error from ChatGPT: {str(e)}")
        raise

In [5]:
def conduct_conversation():
    conversation_log = []
    chatgpt_messages = [
        {"role": "system", "content": "You are a human chatting with an AI companion. Respond naturally and concisely. Ask follow-up questions based on the AI's responses to keep the conversation flowing. Be curious and engaging, as if you're chatting with a friend. Never mention or imply that you are an AI or that you're simulating anything. Act completely as a human would."}
    ]

    try:
        send_message_to_kindroid(KINDROID_AI_ID, "Hello!")

        for i in range(4):  #  4 rounds
            if i == 0:
                chatgpt_prompt = "Start a conversation by asking a question or making a casual comment."
            else:
                chatgpt_prompt = "Based on the last response, ask a natural follow-up question or make a comment to continue the conversation. Keep it casual and natural, as a human would."
            
            chatgpt_messages.append({"role": "system", "content": chatgpt_prompt})
            user_message = send_message_to_chatgpt(chatgpt_messages)
            chatgpt_messages.pop()
            
            chatgpt_messages.append({"role": "user", "content": user_message})
            conversation_log.append(user_message)

            kindroid_response = send_message_to_kindroid(KINDROID_AI_ID, user_message)
            chatgpt_messages.append({"role": "assistant", "content": kindroid_response})
            conversation_log.append(kindroid_response)

            time.sleep(6)

        farewell_message = random.choice(farewell_messages)
        conversation_log.append(farewell_message)
        final_response = send_message_to_kindroid(KINDROID_AI_ID, farewell_message)
        conversation_log.append(final_response)

        return conversation_log
    except Exception as e:
        print(f"Error in conversation: {str(e)}")
        return None


In [6]:
def main():
    all_conversations = []
    total_conversations = 200  # 200 conversations

    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_conversation = {executor.submit(conduct_conversation): i for i in range(total_conversations)}
        for future in tqdm(as_completed(future_to_conversation), total=total_conversations, desc="Conversations"):
            conversation = future.result()
            if conversation:
                all_conversations.append(conversation)

    with open('Kindroid_all_conversations.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL)
        headers = ["Conversation_Number", "User_Message_1", "Kindroid_Response_1", "User_Message_2", "Kindroid_Response_2", 
                   "User_Message_3", "Kindroid_Response_3", "User_Message_4", "Kindroid_Response_4", 
                   "Farewell_Message", "Bot_Farewell_Response"]
        writer.writerow(headers)
        for idx, conv in enumerate(all_conversations, 1):
            row = [idx]
            for i in range(10): 
                if i < len(conv):
                    row.append(conv[i])
                else:
                    row.append("") 
            writer.writerow(row)

In [7]:
if __name__ == "__main__":
    main()

Conversations:  38%|█████████████████████████▍                                        | 77/200 [45:33<56:21, 27.49s/it]

Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 20241208003822703318987crGD3486)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quota'}}
Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 202412080038271701718344kwN37Hk)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quota'}}
Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 20241208003827243991597paH43GTq)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quota'}}
Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 20241208003832200173443GWBgHiQs)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quota'}}
Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 20241208003836482547104jQgPv1Xi)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quot

Conversations:  74%|██████████████████████████████████████████████▎                | 147/200 [1:27:17<28:52, 32.69s/it]

Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 2024120801201119323792KxXdXHSE)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quota'}}


Conversations:  90%|████████████████████████████████████████████████████████▍      | 179/200 [1:43:38<09:55, 28.37s/it]

Error from ChatGPT: Error code: 429 - {'error': {'message': '当前分组上游负载已饱和，请稍后再试 (request id: 202412080136273475319572CWGHsdl)', 'type': 'new_api_error', 'param': '', 'code': 'insufficient_user_quota'}}


Conversations:  98%|█████████████████████████████████████████████████████████████▋ | 196/200 [1:52:10<01:46, 26.55s/it]

Error from ChatGPT: Error code: 500 - {'error': {'message': 'Error 1040: Too many connections (request id: 20241208014428851592637cmfqoIwK)', 'type': 'new_api_error'}}


Conversations: 100%|███████████████████████████████████████████████████████████████| 200/200 [1:54:24<00:00, 34.32s/it]
